In [4]:
import os
import sys
import tensorflow as tf
import keras
import numpy as np
import sklearn
import json

In [5]:
if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive")
    BASE_PATH = "/content/drive/MyDrive/classification-of-medical-images-using-cnn/"
else:
    BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
TEST_DIR = os.path.join(BASE_PATH, "data", "test")
MODEL_PATH = os.path.join(BASE_PATH, "models", "xray_images.keras")
METRICS_PATH = os.path.join(BASE_PATH, "results", "metrics.json")

os.makedirs(os.path.dirname(METRICS_PATH), exist_ok=True)

In [7]:
test_data = keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=(224, 224),
    batch_size=32,
    label_mode="binary",
    shuffle=False,
)

normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

test_data = test_data.map(lambda x, y: (normalization_layer(x), y))

test_data = test_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 624 files belonging to 2 classes.


In [8]:
model = keras.models.load_model(MODEL_PATH)

metrics = model.evaluate(test_data, return_dict=True)

print(json.dumps(metrics, indent=4))

20/20 ━━━━━━━━━━━━━━━━━━━━ 117s 5s/step - accuracy: 0.7980 - loss: 0.4899
{
    "accuracy": 0.8365384340286255,
    "loss": 0.3980506956577301
}


In [11]:
y_true = []
y_scores = []

for images, labels in test_data:
    predictions = model.predict(images)

    y_true.extend(labels.numpy())
    y_scores.extend(predictions.flatten())

y_true = np.array(y_true)
y_scores = np.array(y_scores)

y_pred = (y_scores > 0.5).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


In [ ]:
metrics["precision"] = sklearn.metrics.precision_score(y_true, y_pred)

confusion_matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)

TN, FP, FN, TP = confusion_matrix.ravel().tolist()

metrics["confusion_matrix"] = {
    "TN": int(TN),
    "FP": int(FP),
    "FN": int(FN),
    "TP": int(TP),
}

metrics["f1_score"] = sklearn.metrics.f1_score(y_true, y_pred)
metrics["sensibility"] = TP / (TP + FN)
metrics["specificity"] = TN / (TN + FP)

In [13]:
with open(METRICS_PATH, "w") as f:
    json.dump(metrics, f, indent=4)

print(json.dumps(metrics, indent=4))

{
    "accuracy": 0.8365384340286255,
    "loss": 0.3980506956577301,
    "precision": 0.8618090452261307,
    "confusion_matrix": {
        "TN": 179,
        "FP": 55,
        "FN": 47,
        "TP": 343
    },
    "f1_score": 0.8705583756345178,
    "sensibility": 0.8794871794871795,
    "specificity": 0.7649572649572649
}
